In [1]:
# this code ensure to webscrap the URLs that contains the 
# base part 'https://www.ihg.com/intercontinental/hotels/us/en/'
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from fpdf import FPDF
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
import random
from urllib.parse import urljoin, urlparse

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set a static user agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

# Initialize the Chrome driver with ChromeDriverManager
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Define the base URL part of the dynamic website
base_url_part = 'https://parklane.intercontinental.com/' #'https://www.ihg.com/intercontinental/hotels/us/en/'
url = 'https://parklane.intercontinental.com/' #'https://www.ihg.com/intercontinental/hotels/us/en/reservation'

# Function to set a random delay
def random_delay(min_delay=1, max_delay=3):
    time.sleep(random.uniform(min_delay, max_delay))

# Function to scrape a page and find all links
def scrape_page_and_find_links(page_url):
    driver.get(page_url)
    random_delay(2, 5)
    wait = WebDriverWait(driver, 20)
    try:
        dynamic_content = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        page_data = dynamic_content.text
        print(f"Scraped data from {page_url}")
    except Exception as e:
        print(f"Error loading {page_url}: {e}")
        return "", []
    
    # Find all links on the page
    links = []
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a')
        for element in elements:
            href = element.get_attribute('href')
            if href and href.startswith(base_url_part):
                full_url = urljoin(page_url, href)
                if full_url not in links:
                    links.append(full_url)
    except Exception as e:
        print(f"Error fetching links from {page_url}: {e}")
    
    return page_data, links

# Recursive function to scrape all pages
def recursive_scrape(urls, visited, all_data, depth=0):
    new_urls = []
    
    for url in urls:
        if url not in visited:
            visited.add(url)
            page_data, links = scrape_page_and_find_links(url)
            all_data += f"Page URL: {url}\n{page_data}\n\n"
            new_urls.extend(links)
            
            # Save data periodically
            if depth % 5 == 0:  # Adjust the frequency as needed
                save_to_pdf(all_data, "scraped_data.pdf")
    
    if new_urls:
        # Recursively scrape new URLs
        all_data = recursive_scrape(new_urls, visited, all_data, depth + 1)
    
    return all_data

# Function to save data to PDF
def save_to_pdf(data, file_path):
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Web Scraped Data', 0, 1, 'C')

        def chapter_title(self, title):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, title, 0, 1, 'L')
            self.ln(10)

        def chapter_body(self, body):
            self.set_font('Arial', '', 12)
            # Ensure text is properly encoded to handle Unicode characters
            encoded_body = body.encode('latin-1', 'replace').decode('latin-1')
            self.multi_cell(0, 10, encoded_body)
            self.ln()

    pdf = PDF()
    pdf.add_page()
    pdf.chapter_title('Main Page and Subpages Data')
    pdf.chapter_body(data)
    pdf.output(file_path)

# Initialize the recursive scraping
visited_urls = set()
initial_data, initial_links = scrape_page_and_find_links(url)
all_data = f"Main Page URL: {url}\n{initial_data}\n\n"

# Start recursive scraping with initial links
all_data = recursive_scrape(initial_links, visited_urls, all_data)

# Save the final data to PDF
save_to_pdf(all_data, "scraped_data.pdf")

# Close the browser
driver.quit()

print(f"Data has been scraped and saved to scraped_data.pdf")


d:\ML_DS_DEVOP\PlaceBasedIRS_V2\PlaceBasedIRS\venv\lib\site-packages\fpdf\__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/reservation
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/reservation


C:\Users\Hafeez\AppData\Local\Temp\ipykernel_9300\3215926457.py:89: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
C:\Users\Hafeez\AppData\Local\Temp\ipykernel_9300\3215926457.py:90: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Web Scraped Data', 0, 1, 'C')
C:\Users\Hafeez\AppData\Local\Temp\ipykernel_9300\3215926457.py:93: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
C:\Users\Hafeez\AppData\Local\Temp\ipykernel_9300\3215926457.py:94: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, title, 0, 1, 'L')
C:\Users\Hafeez\AppData\Local\Temp\ipykernel_9300\3215926457.py:98: Dep

Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/stay-mgmt/ManageYourStay
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/reservation#CarouselHotel
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/portsmouth/dompr/hoteldetail
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/boston/bosha/hoteldetail
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/berlin/berha/hoteldetail
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/maamunagau-island/mlemm/hoteldetail
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/stay-mgmt/
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/portsmouth/dompr/hoteldetail/concierge
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/portsmouth/dompr/hoteldetail/meetings-events
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/portsmouth/dompr/hoteldetail/hotel-packages
Scraped data fr

In [9]:
# time to time save in pdf

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from fpdf import FPDF
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
import random
from urllib.parse import urljoin, urlparse

# Setup Chrome options
chrome_options = Options()
# Uncomment the following line if you want to run the browser in headless mode
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set a static user agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

# Initialize the Chrome driver with ChromeDriverManager
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Define the URL of the dynamic website
url = 'https://www.ihg.com/intercontinental/hotels/us/en/reservation'

# Get the domain of the initial URL
initial_domain = urlparse(url).netloc

# Function to set a random delay
def random_delay(min_delay=1, max_delay=3):
    time.sleep(random.uniform(min_delay, max_delay))

# Function to scrape a page and find all links
def scrape_page_and_find_links(page_url):
    driver.get(page_url)
    random_delay(2, 5)
    wait = WebDriverWait(driver, 20)
    try:
        dynamic_content = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        page_data = dynamic_content.text
        print(f"Scraped data from {page_url}")
    except Exception as e:
        print(f"Error loading {page_url}: {e}")
        return "", []
    
    # Find all links on the page
    links = []
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a')
        for element in elements:
            href = element.get_attribute('href')
            if href and href.startswith('http'):
                full_url = urljoin(page_url, href)
                link_domain = urlparse(full_url).netloc
                # Ensure the link is within the same domain
                if link_domain == initial_domain and full_url not in links:
                    links.append(full_url)
    except Exception as e:
        print(f"Error fetching links from {page_url}: {e}")
    
    return page_data, links

# Recursive function to scrape all pages
def recursive_scrape(urls, visited, all_data, depth=0):
    new_urls = []
    
    for url in urls:
        if url not in visited:
            visited.add(url)
            page_data, links = scrape_page_and_find_links(url)
            all_data += f"Page URL: {url}\n{page_data}\n\n"
            new_urls.extend(links)
            
            # Save data periodically
            if depth % 5 == 0:  # Adjust the frequency as needed
                save_to_pdf(all_data, "scraped_data.pdf")
    
    if new_urls:
        # Recursively scrape new URLs
        all_data = recursive_scrape(new_urls, visited, all_data, depth + 1)
    
    return all_data

# Function to save data to PDF
def save_to_pdf(data, file_path):
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Web Scraped Data', 0, 1, 'C')

        def chapter_title(self, title):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, title, 0, 1, 'L')
            self.ln(10)

        def chapter_body(self, body):
            self.set_font('Arial', '', 12)
            # Ensure text is properly encoded to handle Unicode characters
            encoded_body = body.encode('latin-1', 'replace').decode('latin-1')
            self.multi_cell(0, 10, encoded_body)
            self.ln()

    pdf = PDF()
    pdf.add_page()
    pdf.chapter_title('Main Page and Subpages Data')
    pdf.chapter_body(data)
    pdf.output(file_path)

# Initialize the recursive scraping
visited_urls = set()
initial_data, initial_links = scrape_page_and_find_links(url)
all_data = f"Main Page URL: {url}\n{initial_data}\n\n"

# Start recursive scraping with initial links
all_data = recursive_scrape(initial_links, visited_urls, all_data)

# Save the final data to PDF
save_to_pdf(all_data, "scraped_data.pdf")

# Close the browser
driver.quit()

print(f"Data has been scraped and saved to scraped_data.pdf")


Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/reservation
Scraped data from https://www.ihg.com/hotels/us/en/stay-mgmt/ManageYourStay


C:\Users\Hafeez\AppData\Local\Temp\ipykernel_5404\1549676192.py:95: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
C:\Users\Hafeez\AppData\Local\Temp\ipykernel_5404\1549676192.py:96: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Web Scraped Data', 0, 1, 'C')
C:\Users\Hafeez\AppData\Local\Temp\ipykernel_5404\1549676192.py:99: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
C:\Users\Hafeez\AppData\Local\Temp\ipykernel_5404\1549676192.py:100: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, title, 0, 1, 'L')
C:\Users\Hafeez\AppData\Local\Temp\ipykernel_5404\1549676192.py:104: D

Scraped data from https://www.ihg.com/rewardsclub/us/en/join
Scraped data from https://www.ihg.com/rewardsclub/us/en/account-mgmt/home
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/reservation
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/stay-mgmt/ManageYourStay
Scraped data from https://www.ihg.com/intercontinental/content/us/en/meetings-events
Scraped data from https://www.ihg.com/intercontinental/content/us/en/offers
Scraped data from https://www.ihg.com/intercontinental/content/us/en/hotels-resorts
Scraped data from https://www.ihg.com/intercontinental/content/us/en/residences
Scraped data from https://www.ihg.com/intercontinental/content/us/en/loyalty
Scraped data from https://www.ihg.com/intercontinental/content/us/en/offers/summer-soiree?cm_sp=WEB-_-6C-_-AMER-_-EN-_-REV-_-LandLAMERSummerCampaign2024-_-BNR
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/reservation#CarouselHotel
Scraped data from https://www.ihg.com

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.128)
Stacktrace:
	GetHandleVerifier [0x0068C203+27395]
	(No symbol) [0x00623E04]
	(No symbol) [0x00521B7F]
	(No symbol) [0x004FE483]
	(No symbol) [0x0058A06F]
	(No symbol) [0x0059C3D6]
	(No symbol) [0x00583736]
	(No symbol) [0x00557541]
	(No symbol) [0x005580BD]
	GetHandleVerifier [0x00943AB3+2876339]
	GetHandleVerifier [0x00997F7D+3221629]
	GetHandleVerifier [0x0070D674+556916]
	GetHandleVerifier [0x0071478C+585868]
	(No symbol) [0x0062CE44]
	(No symbol) [0x00629858]
	(No symbol) [0x006299F7]
	(No symbol) [0x0061BF4E]
	BaseThreadInitThunk [0x75EAFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x779280CE+286]
	RtlGetAppContainerNamedObjectPath [0x7792809E+238]


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from fpdf import FPDF
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import random
from urllib.parse import urljoin, urlparse

# Setup Chrome options
chrome_options = Options()
# Uncomment the following line if you want to run the browser in headless mode
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Use a fake user agent
ua = UserAgent()
user_agent = ua.random
chrome_options.add_argument(f'user-agent={user_agent}')

# Initialize the Chrome driver with ChromeDriverManager
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Define the URL of the dynamic website
url = 'https://www.ihg.com/intercontinental/hotels/us/en/reservation'

# Get the domain of the initial URL
initial_domain = urlparse(url).netloc

# Function to set a random delay
def random_delay(min_delay=1, max_delay=3):
    time.sleep(random.uniform(min_delay, max_delay))

# Function to scrape a page and find all links
def scrape_page_and_find_links(page_url):
    driver.get(page_url)
    random_delay(2, 5)
    wait = WebDriverWait(driver, 20)
    try:
        dynamic_content = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        page_data = dynamic_content.text
        print(f"Scraped data from {page_url}")
    except Exception as e:
        print(f"Error loading {page_url}: {e}")
        return "", []
    
    # Find all links on the page
    links = []
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a')
        for element in elements:
            href = element.get_attribute('href')
            if href and href.startswith('http'):
                full_url = urljoin(page_url, href)
                link_domain = urlparse(full_url).netloc
                # Ensure the link is within the same domain
                if link_domain == initial_domain and full_url not in links:
                    links.append(full_url)
    except Exception as e:
        print(f"Error fetching links from {page_url}: {e}")
    
    return page_data, links

# Recursive function to scrape all pages
def recursive_scrape(urls, visited):
    all_data = ""
    new_urls = []
    
    for url in urls:
        if url not in visited:
            visited.add(url)
            page_data, links = scrape_page_and_find_links(url)
            all_data += f"Page URL: {url}\n{page_data}\n\n"
            new_urls.extend(links)
    
    if new_urls:
        # Recursively scrape new URLs
        all_data += recursive_scrape(new_urls, visited)
    
    return all_data

# Initialize the recursive scraping
visited_urls = set()
initial_data, initial_links = scrape_page_and_find_links(url)
all_data = f"Main Page URL: {url}\n{initial_data}\n\n"

# Start recursive scraping with initial links
all_data += recursive_scrape(initial_links, visited_urls)

# Close the browser
driver.quit()

# Create a PDF file with Unicode support
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Web Scraped Data', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(10)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        # Ensure text is properly encoded to handle Unicode characters
        encoded_body = body.encode('latin-1', 'replace').decode('latin-1')
        self.multi_cell(0, 10, encoded_body)
        self.ln()

pdf = PDF()
pdf.add_page()
pdf.chapter_title('Main Page and Subpages Data')
pdf.chapter_body(all_data)

pdf_file_path = "scraped_data.pdf"
pdf.output(pdf_file_path)

print(f"Data has been scraped and saved to {pdf_file_path}")


Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/reservation
Scraped data from https://www.ihg.com/hotels/us/en/stay-mgmt/ManageYourStay
Scraped data from https://www.ihg.com/rewardsclub/us/en/join
Scraped data from https://www.ihg.com/rewardsclub/us/en/account-mgmt/home
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/reservation
Scraped data from https://www.ihg.com/intercontinental/hotels/us/en/stay-mgmt/ManageYourStay
Scraped data from https://www.ihg.com/intercontinental/content/us/en/meetings-events
Scraped data from https://www.ihg.com/intercontinental/content/us/en/offers
Scraped data from https://www.ihg.com/intercontinental/content/us/en/hotels-resorts
Scraped data from https://www.ihg.com/intercontinental/content/us/en/residences
Scraped data from https://www.ihg.com/intercontinental/content/us/en/loyalty
Scraped data from https://www.ihg.com/intercontinental/content/us/en/offers/summer-soiree?cm_sp=WEB-_-6C-_-AMER-_-EN-_-REV-_-LandLAMER

KeyboardInterrupt: 